# Creating dataset

# Creating classification model

In [1]:
import os
import pickle
import numpy as np
import os
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import metrics
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
import keras
import shutil
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

Using TensorFlow backend.


In [2]:
DATASET_PATH = '../../datasets/Classification/Dataset - 7 dias'
CATEGORIES = ["Normal", "Anormal", "Morta"]

##### Read dataset

In [3]:
IMG_SIZE = 256
NUM_CLASSES = 3
def read_dataset():
    images_raw = open(os.path.join(DATASET_PATH, "7_days_images.pickle" ), "rb")
    images = pickle.load(images_raw)

    labels_raw = open(os.path.join(DATASET_PATH, "7_days_labels.pickle" ), "rb")
    labels = pickle.load(labels_raw)

    images = np.array(images)
    labels = np.array(labels)
    
    return images, labels

###### Turn it into a binary classification problem

In [4]:
def to_binary(labels):
    labels[labels == 2] = 1
    return labels
    

###### Train Test split

In [5]:
def count_samples(train_labels, test_labels):
    print("In training:")
    for i in range(0,NUM_CLASSES):
        print(f'Class {i}: {round(100 * np.count_nonzero(np.array(train_labels) == i) / len(train_labels), 2)}%')
    print("In test:")
    for i in range(0,NUM_CLASSES):
        print(f'Class {i}: {round(100 * np.count_nonzero(np.array(test_labels) == i) / len(test_labels), 2)}%')


def train_test_split(images, labels):

    from sklearn.model_selection import train_test_split

    train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size=0.1, stratify=labels)
    
    print("In training:")
    for i in range(0,len(np.unique(labels))):
        print(f'Class {i}: {round(100 * np.count_nonzero(np.array(train_labels) == i) / len(train_labels), 2)}%')
    print("In test:")
    for i in range(0,len(np.unique(labels))):
        print(f'Class {i}: {round(100 * np.count_nonzero(np.array(test_labels) == i) / len(test_labels), 2)}%')
        
    return train_images, test_images, train_labels, test_labels

#### Ajustando o dataset desbalanceado

###### Data augmentation -> data generator

In [6]:
# data augmentation with ImageDataGenerator -> did not go well
# from keras.preprocessing.image import ImageDataGenerator
# datagen = ImageDataGenerator(
#     featurewise_center=True,
#     featurewise_std_normalization=False,
#     rotation_range=20,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
#     horizontal_flip=True,
#     validation_split=0.25
# )
# datagen.fit(train_images)

# # it = datagen.flow(train_images, batch_size=1, save_to_dir="test", shuffle= True)

# # # generate samples and plot
# # for i in range(50):
# #      # generate batch of images
# #     batch = it.next()
# #     # convert to unsigned integers for viewing
# #     image = batch[0].astype('uint8')
# #     # plot raw pixel data
# #     plt.imshow(image, cmap="gray")
# #     plt.show()
# # show the figure

###### Weighted Classes 

In [7]:
# Trying class weights
# it does not work with one hot encoded data
# from sklearn.utils import class_weight
# class_weights = class_weight.compute_class_weight('balanced',
#                                                  np.unique(train_labels),
#                                                  train_labels)

def get_sample_weights(train_labels):
    from sklearn.utils import compute_sample_weight
    
    return compute_sample_weight('balanced', train_labels)  #->>>> remember to add the weights in model.fit

###### SMOTE

In [8]:
# classes, counts = np.unique(train_labels, return_counts=True)
# print(f"The number of elements for each class in training now are\nClass 0: {counts[0]}\nClass 1 {counts[1]}\nClass 2 {counts[2]}\n")

In [9]:
# from imblearn.over_sampling import SMOTE
# sm = SMOTE(random_state=42)

# train_images_shape = train_images.shape
# train_images = train_images.reshape((train_images_shape[0], IMG_SIZE * IMG_SIZE * 3))
# train_images, train_labels = sm.fit_resample(train_images, train_labels)


In [10]:
# train_images_shape = train_images.shape
# train_images = train_images.reshape((train_images_shape[0], IMG_SIZE, IMG_SIZE, 3))


In [11]:
# classes, counts = np.unique(train_labels, return_counts=True)
# print(f"The number of elements for each class in training now are\nClass 0: {counts[0]}\nClass 1 {counts[1]}\nClass 2 {counts[2]}\n")

##### One hot label encoding

In [12]:
def to_one_hot(data):
    import pandas as pd
    data = pd.get_dummies(data)
    data = pd.DataFrame.to_numpy(data)
    return data

##### Normalizing training images

In [13]:
def normalize_images(data):
    data = tf.keras.utils.normalize(data, axis=0, order=2)
    return data

### Creating the model


In [14]:
EPOCHS = None
TRAIN_TEST_SPLIT = None
BATCH_SIZE = None
# print((1 - TRAIN_TEST_SPLIT) * train_images.shape[0],BATCH_SIZE)

METRICDIR = '../metricas/classificacao/7_d/tests-binary'

MODELDIR = os.path.join(METRICDIR, "adhoc")

CHECKPOINT_DIR = os.path.join(MODELDIR, "models") 

if os.path.exists(MODELDIR):
    shutil.rmtree(MODELDIR)
    os.makedirs(CHECKPOINT_DIR)
    
my_metrics = ["accuracy",
       tf.keras.metrics.Precision(),
       tf.keras.metrics.Recall(),
       tf.keras.metrics.AUC(),
       tf.keras.metrics.TruePositives(),
       tf.keras.metrics.TrueNegatives(),
       tf.keras.metrics.FalsePositives(),
       tf.keras.metrics.FalseNegatives(),]

In [15]:
def create_model():
       
    model = Sequential()

    model.add(   Conv2D(8, (3,3), input_shape=(IMG_SIZE,IMG_SIZE, 3))  )
    model.add(Dropout(rate=0.3))
    model.add(Activation("relu"))

    model.add(Conv2D(6, (3,3)))
    model.add(Dropout(rate=0.3))
    model.add(Activation("relu"))
    
    model.add(Conv2D(2, (3,3)))
    model.add(Dropout(rate=0.4))
    model.add(Activation("relu"))
    
    model.add(Flatten())
    model.add(Dense(3))
    
    model.add(Activation("sigmoid"))
    
    opt = tf.keras.optimizers.Adam()
    model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=my_metrics)
    
    return model

def get_model_name(k):
    return 'model_'+str(k)

def plot_metric(history,metric_name, chart_name, save_dir, save = False, show = False):
    plt.cla() 
    plt.plot(history.history[metric_name], label = metric_name + ' (training data)')
    plt.plot(history.history['val_' + metric_name], label = metric_name + ' (validation data)')
    plt.title(chart_name)
    plt.ylabel( metric_name + ' value')
    plt.xlabel('No. epoch')
    plt.legend(loc="upper left")
    if save:
        plt.savefig(os.path.join(save_dir, metric_name))
    if show:
        plt.show()
        
def plot_cm(cf_matrix, save_dir, save):
    
    import seaborn as sns; sns.set_theme()
   
    group_names = ['True Pos','False Pos','False Neg','True Neg']
    group_counts = ["{0:0.0f}".format(value) for value in
                    cf_matrix.flatten()]
    group_percentages = ["{0:.2%}".format(value) for value in
                         cf_matrix.flatten()/np.sum(cf_matrix)]
    labels = [f"{v1}\n{v2}\n{v3}" for v1, v2, v3 in
              zip(group_names,group_counts,group_percentages)]
    labels = np.asarray(labels).reshape(2,2)
    
    plt.cla() 
    
    sns.heatmap(cf_matrix, annot=labels, fmt='', cmap='Blues')
    if save:
        plt.savefig(os.path.join(save_dir, "confusion matrix"))
    plt.show()

##### Testing the model

In [16]:
def one_hot_to_1d(test_labels):
    
    test_labels_1d = []
    
    for label in test_labels:
        print()
        if (label == [1, 0]).all():
            test_labels_1d.append(0)
        elif (label == [0, 1]).all():
            test_labels_1d.append(1)
        else:
            test_labels_1d.append(2)
    
    return test_labels_1d
    
def get_predictions(model,test_images, test_labels):
    prediction = model.predict(test_images, batch_size=1)

    return tf.argmax(prediction, axis=-1)

In [17]:
def train_models(train_images, train_labels, num_folds=10):
    
    from sklearn.model_selection import StratifiedKFold
    
    kfold = StratifiedKFold(n_splits=num_folds)

    #metrics containers
    acc = []
    prec = []
    rec = []
    auc = []

    # K-fold Cross Validation model evaluation
    fold_no = 1
    for train, test in kfold.split(train_images, train_labels):

        count_samples(train_labels[train], train_labels[test])
        
#         from tf.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

        if fold_no - 1 == 0:
            idx_str = ""
        else:
            idx_str = f"_{fold_no}"
            
        fold_checkpoint_dir = os.path.join(CHECKPOINT_DIR, get_model_name(fold_no))
        
        if not os.path.exists(fold_checkpoint_dir):
            os.makedirs(fold_checkpoint_dir)
        
#         my_callbacks = [
# #             ModelCheckpoint(os.path.join(fold_checkpoint_dir,"model.h5"), monitor=f"val_precision", verbose=1, save_best_only=True, mode='max', save_freq='epoch'),
#             ReduceLROnPlateau(monitor=f'loss', factor=0.7, patience=2, min_lr=0.00000000001, mode='min'),
#         ]

        model = create_model()

        # Generate a print
        print('------------------------------------------------------------------------', end="\n\n")
        print(f'Training for fold {fold_no} ...')

#         # Fit data to model
#         history = model.fit(
#             train_images[train],
#             train_labels[train],
#             batch_size = BATCH_SIZE,
#             epochs = EPOCHS,
#             validation_split=TRAIN_TEST_SPLIT,
#             sample_weight=get_sample_weights(train_labels[train]),
#             callbacks=[my_callbacks])
        # Fit data to model
        history = model.fit(
            train_images[train],
            keras.utils.to_categorical(train_labels[train]),
            batch_size = BATCH_SIZE,
            epochs = EPOCHS,
            validation_split=TRAIN_TEST_SPLIT,
            sample_weight=get_sample_weights(train_labels[train]))
        
        thresh = 0.5
        
        prediction = model.predict(train_images[test], batch_size=1)
        prediction[prediction > thresh] = 1
        prediction[prediction <= thresh] = 0
        
        cm = metrics.confusion_matrix(train_labels[test],prediction )
        
        plot_cm(cm, save_dir=fold_checkpoint_dir, save=True)
        
        #visualizing training
        NUM_METRICS = len(my_metrics) + 1
        for metric in list(history.history.keys())[:NUM_METRICS]:
            plot_metric(history, metric, metric + " for soybean classification", save_dir=fold_checkpoint_dir, save=True)

        # Generate generalization metrics
        scores = model.evaluate(train_images[test], train_labels[test], verbose=0)
        print(f'Scores for fold {fold_no}:')
        for i in range(len(my_metrics)):   
            print(f'{model.metrics_names[i]} of {scores[i]}')

        acc.append(scores[1])
        prec.append(scores[2])
        rec.append(scores[3])
        auc.append(scores[4])

        # Increase fold number
        fold_no = fold_no + 1
        
    return acc, prec, rec, auc

##### Training

In [18]:
images, labels = read_dataset()

images = normalize_images(images)
labels = to_binary(labels)

In [20]:
EPOCHS = 10
TRAIN_TEST_SPLIT = 0.25
BATCH_SIZE = 32
acc, prec, rec, auc = train_models(images, labels)

In training:
Class 0: 78.97%
Class 1: 21.03%
Class 2: 0.0%
In test:
Class 0: 78.99%
Class 1: 21.01%
Class 2: 0.0%
------------------------------------------------------------------------

Training for fold 1 ...


ValueError: A target array with shape (1065, 2) was passed for an output of shape (None, 3) while using as loss `categorical_crossentropy`. This loss expects targets to have the same shape as the output.

In [ ]:
print("Acurácia média: ",np.mean(acc), np.std(acc))
print("Precisão média: ",np.mean(prec), np.std(prec))
print("Recall médio: ",np.mean(rec), np.std(rec))
print("Auc média: ",np.mean(auc), np.std(auc))

In [ ]:
for model in os.listdir("../metricas/classificacao/7_d/tests-binary/adhoc/models"):
    metric_path = os.path.join("../metricas/classificacao/7_d/tests-binary/adhoc/models", model, "loss.png")
    img = plt.imread(metric_path)
    plt.imshow(img)
    plt.show()